# <span style="font-family:Papyrus; font-size:4em; color:DarkOrange;">Odei</span>

## **Things to Remember:**

1) Read the getting started before exploring this write-up.

2) All file paths shown are based on the computer used in this write-up.

3) Use the Resource page/pdf to see a list all websites and programs used in this write-up.

## <font color=Darkorange>Odei 1</font>

On the morning of August 15th 2018, there was a spike in network traffic. Check your network resources. What is the IP address of the requester?  

### Solution:

First, take a look at the pcap files for August 15, 2018:

In [1]:
Get-ChildItem -path "..\pcaps\" -Recurse -Filter "*2018-08-15*"



    Directory: C:\Users\Administrator\Artifacts\pcaps


Mode                LastWriteTime         Length Name                                                                  
----                -------------         ------ ----                                                                  
-a----        8/23/2018   3:32 PM      100003270 2018-08-15_06-22-41.pcap                                              
-a----        8/23/2018   3:33 PM      100002467 2018-08-15_09-04-14.pcap                                              
-a----        8/23/2018   3:33 PM      100005890 2018-08-15_09-27-04.pcap                                              
-a----        8/23/2018   3:34 PM      100017730 2018-08-15_09-49-37.pcap                                              
-a----        8/23/2018   3:34 PM      100000883 2018-08-15_10-11-46.pcap                                              
-a----        8/23/2018   3:35 PM      100000004 2018-08-15_10-51-33.pcap                             

Let's merge the pcap files to make filtering easier:

In [1]:
Set-Alias -Name merge -Value "C:\Program Files\Wireshark\mergecap.exe" | Out-Null
Set-Alias -Name tshark -Value "C:\Program Files\Wireshark\tshark.exe" | Out-Null

merge -F pcap -w ..\pcaps\8_15_morning_merge.pcap ..\pcaps\2018-08-15_06-22-41.pcap ..\pcaps\2018-08-15_09-04-14.pcap ..\pcaps\2018-08-15_09-27-04.pcap ..\pcaps\2018-08-15_09-49-37.pcap ..\pcaps\2018-08-15_10-11-46.pcap ..\pcaps\2018-08-15_10-51-33.pcap
write-host "Complete"

Complete


Open the merged morning pcap file in Wireshark. The question is asking for the IP of the requester that caused a spike in network traffic. Wireshark has a feature called conversations under the statistics tab. In the context of the question, conversations are important because the requester IP seems to have caused a spike in traffic. Conversations allows you to analyze the total packet transfers between two IP addresses. 

![Odei](pics/Odei1-1.png)

Conversations has multiple tabs, but by looking at the number of packets in the IPv4 and IPv6 tabs, it can be deduced that this attack probably used IPv4 because of the large number of items inside it.

Inside the IPv4 tab, filter the content by largest packets. This will organize the data to show which conversation had the most packets transfered and in turn help show what would have caused a spike in traffic. 

![Odei](pics/Odei1-2.png)

The largest amount of packets was transfered between Address A (5.101.80.28) and Address B (192.168.1.23). Referencing the network diagram for the competition, Address B (192.168.1.23) is the Orko Billpayer Site (Windows Server 2016). That leaves Address A (5.101.80.28) as the answer. 

#### Answer: 5.101.80.28 

## <font color=Darkorange>Odei 2</font>

What internal IP address was being consistently visited?

### Solution:

Continue in the same merged morning pcap file. Go to **statistics>Endpoints**. In Endpoints click **IPv4** and organize it by **Bytes**.

![Odei](pics/Odei2-1.png)

Notice the IP address with the most visits is **192.168.1.23**.

#### Answer: 192.168.1.23

## <font color=Darkorange>Odei 3</font>

What is the external IP of this website?

### Solution:

From Odei 1, its known that the source IP Address is **5.101.80.28** and from the question its known that this source IP address is probably requesting data from this external IP. Continue in the merged morning pcap and filter by **ip.addr == 5.101.80.28 && http.request.method == GET** and by looking specifically at the **http.host** field. 


In [12]:
Set-Alias -Name tshark -Value "C:\Program Files\Wireshark\tshark.exe"
tshark -r "C:\Users\Administrator\Artifacts\pcaps\8_15_morning_merge.pcap" -Y "ip.addr == 5.101.80.28 && http.request.method == GET" -T "fields" -e "http.host" > "C:\Users\Administrator\Artifacts\pcaps\output1.txt"
Get-Content "C:\Users\Administrator\Artifacts\pcaps\output1.txt" -First 10

50.0.0.245
50.0.0.245
50.0.0.245
50.0.0.245
50.0.0.245
50.0.0.245
50.0.0.245
50.0.0.245
50.0.0.245
50.0.0.245




#### Answer: 50.0.0.245

## <font color=Darkorange>Odei 4</font>

The requester tried getting account info for several user names. What is the first username they requested?

### Solution:

From Odei 1, it's known that the requesters IP address is **5.101.80.28**. From the context of the question, it's known that the username was requested by this IP, so it probably used a GET request. 

Continue in the merged morning pcap file and filter again by **ip.addr == 5.101.80.28 && http.request.method == GET.**.

![Odei](pics/Odei3-1.png)

Look in the info column and notice that multiple packets are requesting account info for **blah**.

![Odei](pics/Odei8.png)

#### Answer: blah

## <font color=Darkorange>Odei 5</font>

What is the content type of the returned data from the URL that is consistently hit by the requester?

### Solution:

It's known from Odei 1 that the IP address **5.101.80.28** is being used by the hacker and it can be deduced that this IP is most likely the destination.

![Odei](pics/Odei5-4.png)

Filtering by **ip.dst==5.101.80.28** results in over 72,000 packets, additional filtering is needed. The question is asking for the "content type" of the returned data. Filtering by the IP destination (**ip.dst==5.101.80.28**) and content type (**http.content_type**) results in a little over 6000 packets.

![Odei](pics/Odei5-5.png)

It's known from the question that this content type is being returned by a URL that is consistently hit by the requester. With that knowledge, organize the packets by the **info** column. This displays data a few different contents types, but primary **application/pdf**. 

![Odei](pics/Odei5-6.png)



#### Answer: application/pdf

## <font color=Darkorange>Odei 6</font>

What is the the full request URL to get the PDF for user 1's bill?

### Solution:

Starting from Odei 5, it's already known that PDFs have been uploaded and that these packets have a Request URI where packets are being delivered to.  

![Odei](pics/Odei5-6.png)

Examining the Request URIs from the application/pdfs, notice a full request URL of **http://50.0.0.245/Account/GetBillPdf?billHeaderId=1**.


#### Answer: http://50.0.0.245/Account/GetBillPdf?billHeaderId=1


## <font color=Darkorange>Odei 7</font>

What did the attacker use to exfiltrate the PDFs?

### Solution:

It's known from Odei 1 that the attacker uses the IP **5.101.80.28** and that the full request URL is **http://50.0.0.245/Account/GetBillPdf?billHeaderId=1** to get the PDFs. 

Continue in the merged morning pcap file with the filter **ip.addr==5.101.80.28 && http.request.full_uri == "http://50.0.0.245/Account/GetBillPdf?billHeaderId=1"**. This filters the content down to 11 packets.


![Odei](pics/Odei7-1.png)

The question wants to know what the attacker used to exfiltrate the PDFs. When examining the filtered packets, locate the **user-agent** field under the Hypertext drop-down. This field will list the software (a software agent) that is acting on behalf of a user to send data. A few of the packet have a user agent of Mozilla, which is Firefox. However, the other packets have a user agent of Python-Requests. Knowing that Firefox is used for typical browsing purposes, it can be deduced that **python-requests** is what the attacker used to exfiltrate the PDFs.

![Odei](pics/Odei7-2.png)

#### Answer: python-requests

## <font color=Darkorange>Odei 8</font>

After the attacker exfiltrated the PDFs, the requester visited another URL that allowed them to access more personal information (such as SSN) from all of the users. What is the URL?

### Solution:

It's known from the question that the attacker (5.101.80.28) visited the URL after exfiltrating the PDFs that were listed in Odei 5. 

Continue in the merged morning pcap file and filter by the attackers IP and the GET request method: **ip.addr==5.101.80.28 && http.request.method == GET**. Since the attacker visited this URL after exfiltrating the PDF files, organize the filtered content by the time column.

![Odei](pics/Odei8-1.png)

The only entry past the PDF exfiltrations is **/account/getcustomerinfo**. Locate the URL by clicking on this packet and navigating to the drop-down **Hyptertext Transfer Protocol**. From there the full request URI is **http://50.0.0.245/account/getcustomerinfo**.

![Odei](pics/Odei8-2.png)

Since this get request is made by the attacker after the PDF file exfiltration, it can be deduced that this is the URL the attacker visited. 


#### Answer: http://50.0.0.245/account/getcustomerinfo


## <font color=Darkorange>Odei 9</font>

What data type was returned from the last attack?

### Solution:

Starting from the same packet from Odei 8 with the URL http://50.0.0.245/account/getcustomerinfo, right click and follow the TCP Stream. The TCP stream is needed because it logs streams of data between connections during a transfer. Since the question is asking for the data type that was returned, the stream should provide this information.

![Odei](pics/Odei9-1.png)

The top red text shows the get request from the attacker. The blue text shows what is returned from the host machine. In the blue text there is a field called **content-type**. This field displays what type of data was returned to the host. In this case that field shows **application/json**. Since the question is asking what data type was sent back, it can be deduced that **json** is the data type.

#### Answer: json


## <font color=Darkorange>Odei 10</font>

What is Amaya Labankada’s unique id within the customer data?

### Solution:

Continue in the TCP stream from Odei 9. It's known that the data is formatted in JSON from Odei 9. Scrolling through the TCP stream, you'll notice usernames, email addresses, and much more. Use the find option in the TCP stream window to search for Amaya. This search shows a json package with Amaya's data and a field called **id**. This field contains the ID **ff060f77-5203-4d4e-90b3-8329154cb023**. It can be deduced that this is the unique identifier for Amaya.

![Odei](pics/Odei19.png)

#### Answer:  ff060f77-5203-4d4e-90b3-8329154cb023